In [10]:
'''
Построение двухслойной нейронный сети для классификации цветков ириса
'''

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sklearn здесь только, чтобы разделить выборку на тренировочную и тестовую
from sklearn.model_selection import train_test_split


### Шаг 1. Определение функций, которые понадобяться для обучения
# преобразование массива в бинарный вид результатов
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

# преобразование массива в необходимый вид
def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))

    for i in range(len(Y)):
        l = layer2[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr

# сигмоида и ее производная
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return sigmoid(x)*(1 - sigmoid(x))

# нормализация массива
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

def count_acc(n, k, w0, w1):
    #n = 0.005

    # массив для ошибок, чтобы потом построить график
    errors = []

    # процесс обучения
    for i in range(k):

        # прямое распространение(feed forward)
        layer0 = X_train
        layer1 = sigmoid(np.dot(layer0, w0))
        layer2 = sigmoid(np.dot(layer1, w1))

        # обратное распространение(back propagation) с использованием градиентного спуска
        layer2_error = y_train - layer2
        layer2_delta = layer2_error * sigmoid_deriv(layer2)

        layer1_error = layer2_delta.dot(w1.T)
        layer1_delta = layer1_error * sigmoid_deriv(layer1)

        w1 += layer1.T.dot(layer2_delta) * n
        w0 += layer0.T.dot(layer1_delta) * n

        error = np.mean(np.abs(layer2_error))
        errors.append(error)
        accuracy = (1 - error) * 100
    return accuracy

### Шаг 2. Подготовка тренировочных данных
# получения данных из csv файла. укажите здесь путь к файлу Iris.csv
iris_data = pd.read_csv("D:/КУРСЫ/GB/Нейронные сети/Iris.csv")
print(iris_data.head()) # расскоментируйте, чтобы посмотреть структуру данных

# репрезентация данных в виде графиков
g = sns.pairplot(iris_data.drop("Id", axis=1), hue="Species")
# plt.show() # расскоментируйте, чтобы посмотреть

# замена текстовых значений на цифровые
iris_data['Species'].replace(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor'], [0, 1, 2], inplace=True)

# формирование входных данных
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
x = pd.DataFrame(iris_data, columns=columns)
x = normalize(x.values)

# формирование выходных данных(результатов)
columns = ['Species']
y = pd.DataFrame(iris_data, columns=columns)
y = y.values
y = y.flatten()
y = to_one_hot(y)

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
   
### Шаг 3. Обученние нейронной сети
# присваевание случайных весов
#w0 = 2*np.random.random((4, 5)) - 1 # для входного слоя   - 4 входа, 3 выхода
#w1 = 2*np.random.random((5, 3)) - 1 # для внутреннего слоя - 5 входов, 3 выхода

list_n = [0.1, 0.005, 0.001, 0.0005]
list_k = [10000, 50000, 100000, 150000, 200000, 300000]

for n in list_n:
    for k in list_k:
        print("Скорость обчучения ", n)
        print("Количество итераций ", k)
        w0 = 2*np.random.random((4, 5)) - 1 # для входного слоя   - 4 входа, 3 выхода
        w1 = 2*np.random.random((5, 3)) - 1 # для внутреннего слоя - 5 входов, 3 выхода

        accuracy = count_acc(n, k, w0, w1)
        print("Точность нейронной сети " + str(round(accuracy,2)) + "%")
        
# скорость обучения (learning rate)
#n = 0.005

# массив для ошибок, чтобы потом построить график
#errors = []

# процесс обучения
#for i in range(10000):

    # прямое распространение(feed forward)
#    layer0 = X_train
#    layer1 = sigmoid(np.dot(layer0, w0))
#    layer2 = sigmoid(np.dot(layer1, w1))

    # обратное распространение(back propagation) с использованием градиентного спуска
#    layer2_error = y_train - layer2
#    layer2_delta = layer2_error * sigmoid_deriv(layer2)
    
#    layer1_error = layer2_delta.dot(w1.T)
#    layer1_delta = layer1_error * sigmoid_deriv(layer1)
    
#    w1 += layer1.T.dot(layer2_delta) * n
#    w0 += layer0.T.dot(layer1_delta) * n
    
#    error = np.mean(np.abs(layer2_error))
#    errors.append(error)
#    accuracy = (1 - error) * 100


### Шаг 4. Демонстрация полученных результатов
# черчение диаграммы точности в зависимости от обучения
#plt.plot(errors)
#plt.xlabel('Обучение')
#plt.ylabel('Ошибка')
#plt.show() # расскоментируйте, чтобы посмотреть 
        
#print("Точность нейронной сети " + str(round(accuracy,2)) + "%")

   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa
Скорость обчучения  0.1
Количество итераций  10000
Точность нейронной сети 98.4%
Скорость обчучения  0.1
Количество итераций  50000


C:\Users\user\AppData\Local\Temp\ipykernel_13200\4171736632.py:39: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Точность нейронной сети 97.35%
Скорость обчучения  0.1
Количество итераций  100000
Точность нейронной сети 99.82%
Скорость обчучения  0.1
Количество итераций  150000
Точность нейронной сети 98.28%
Скорость обчучения  0.1
Количество итераций  200000
Точность нейронной сети 99.34%
Скорость обчучения  0.1
Количество итераций  300000
Точность нейронной сети 98.36%
Скорость обчучения  0.005
Количество итераций  10000
Точность нейронной сети 89.71%
Скорость обчучения  0.005
Количество итераций  50000
Точность нейронной сети 96.89%
Скорость обчучения  0.005
Количество итераций  100000
Точность нейронной сети 98.25%
Скорость обчучения  0.005
Количество итераций  150000
Точность нейронной сети 98.23%
Скорость обчучения  0.005
Количество итераций  200000
Точность нейронной сети 98.25%
Скорость обчучения  0.005
Количество итераций  300000
Точность нейронной сети 97.03%
Скорость обчучения  0.001
Количество итераций  10000
Точность нейронной сети 77.59%
Скорость обчучения  0.001
Количество итераций

Видно, что результаты сильно зависят от выбора начальных значений w0, w1. Где то результаты достигли точности 99%, а где то опустились до 72%. Попробуем их зафиксировать. 

In [11]:
w0 = 2*np.random.random((4, 5)) - 1 # для входного слоя   - 4 входа, 3 выхода
w1 = 2*np.random.random((5, 3)) - 1 # для внутреннего слоя - 5 входов, 3 выхода

for n in list_n:
    for k in list_k:
        print("Скорость обчучения ", n)
        print("Количество итераций ", k)
        

        accuracy = count_acc(n, k, w0, w1)
        print("Точность нейронной сети " + str(round(accuracy,2)) + "%")
        

Скорость обчучения  0.1
Количество итераций  10000
Точность нейронной сети 98.5%
Скорость обчучения  0.1
Количество итераций  50000
Точность нейронной сети 99.0%
Скорость обчучения  0.1
Количество итераций  100000
Точность нейронной сети 98.65%
Скорость обчучения  0.1
Количество итераций  150000
Точность нейронной сети 99.98%
Скорость обчучения  0.1
Количество итераций  200000
Точность нейронной сети 99.97%
Скорость обчучения  0.1
Количество итераций  300000


C:\Users\user\AppData\Local\Temp\ipykernel_13200\4171736632.py:39: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Точность нейронной сети 99.94%
Скорость обчучения  0.005
Количество итераций  10000
Точность нейронной сети 99.94%
Скорость обчучения  0.005
Количество итераций  50000
Точность нейронной сети 99.94%
Скорость обчучения  0.005
Количество итераций  100000
Точность нейронной сети 99.94%
Скорость обчучения  0.005
Количество итераций  150000
Точность нейронной сети 99.94%
Скорость обчучения  0.005
Количество итераций  200000
Точность нейронной сети 99.94%
Скорость обчучения  0.005
Количество итераций  300000
Точность нейронной сети 99.93%
Скорость обчучения  0.001
Количество итераций  10000
Точность нейронной сети 99.93%
Скорость обчучения  0.001
Количество итераций  50000
Точность нейронной сети 99.93%
Скорость обчучения  0.001
Количество итераций  100000
Точность нейронной сети 99.93%
Скорость обчучения  0.001
Количество итераций  150000
Точность нейронной сети 99.93%
Скорость обчучения  0.001
Количество итераций  200000
Точность нейронной сети 99.93%
Скорость обчучения  0.001
Количество и

Видно, что при уменьшении скорости обучения, увеличение количество итераций не повышает точность нейронной сети. Кроме того получилось, что при уменьшении скорости обучения, точность в какой то момент начинает снижаться.    